## AutoML
## Empirical Tests - PyCaret

This project aims to explore some of the main **AutoML tools** available, which involves the following tasks:
1. Reading of technical articles concerning the automated machine learning field.
2. Discussion about machine learning pipelines and the automation of some of their components.
3. Identification of the most interesting Python libraries for automatic ML pipeline construction.
4. Quick implementation of the selected tools with simulated data.
5. Careful exploration of the APIs of the selected tools.
6. Comparison among selected tools concerning: model performance, computation time, and usability.

All of these activities derive from the **objectives** of this project, which are: i) reflection about ML pipeline components; ii) discussion and analysis of AutoML tools; iii) identification of key-points of AutoML frameworks; iv) definition of: the advantages and disadvantages of main AutoML tools, and, first of all, the relavance and adequacy of implementing AutoML.

---------------------

In this series of notebooks, we test out different AutoML Python libraries and compare them according to the following criteria: performance metrics of developed pipelines evaluated on test data; computation time (i.e., the performance relative to the available time budget of the search process); and usability of the tool.

* **Performance:** for each tool, after providing them with a training data (that will receive the appropriate validation approach by each tool), and after the search for the best ML pipeline, the selected one will be evaluated on a hold-out dataset (25% of the complete dataset). The model assessment will be based on the following metrics, since the supervised learning task is a binary classification here: [ROC-AUC](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html), [average precision score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html), [Brier score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.brier_score_loss.html), [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html), and [MCC](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html).

* **Computation time:** all tested AutoML tools have some sort of time budget for the search process. Therefore, instead of minimizing the computation time across all tested tools, we will explore three different time budgets: 20 minutes, 1 hour, and 6 hours. Consequently, one of the main aspects of the comparison among tools will be the performance achieved by each one of them given different time budgets, besides of the average performance throughout all time budgets.

* **Usability:** this aspect of the comparison refers to how easy it is to set up the search for each one of the tested tools. Also important are the outputs of the search process, mainly in terms of the visualization and assessment of constructed and selected pipelines. Besides, the diversity of produced information about the search and how clear it is to access and interpret these data are also an aspect to have in mind. Finally, the more straightforward it is to use a selected pipeline the better is the tool.

The empirical tests follow the reading and discussing of the APIs of all selected tools. So, since the main initialization arguments, methods and attributes have been defined, they will be used accordingly in these notebooks.

The data used for the empirical tests was found in Kaggle repository of datasets. It consists of a dataset for binary classification whose objective is to construct a classification algorithm for the [identification of malware apps](https://www.kaggle.com/saurabhshahane/android-permission-dataset). It has 27310 unique instances (mobile phone applications) and 184 variables, among which one is the binary outcome variable and another is the name of the app. Since the main objective of this project is to explore AutoML tools, only some basic feature engineering operations were implemented, besides of a short description and exploration of the data.

------------

**Summary:**
1. [Libraries](#libraries)<a href='#libraries'></a>.
2. [Functions and classes](#functions_classes)<a href='#functions_classes'></a>.
3. [Settings](#settings)<a href='#settings'></a>.
4. [Importing datasets](#imports)<a href='#imports'></a>.
    * [Features and outcome variables](#feats_outcomes)<a href='#feats_outcomes'></a>.


5. [Data description](#data_description)<a href='#data_description'></a>.
6. [ML pipeline](#ml_pipeline)<a href='#ml_pipeline'></a>.

<a id='libraries'></a>

## Libraries

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd "/content/gdrive/MyDrive/Studies/autoML/Codes"

/content/gdrive/MyDrive/Studies/autoML/Codes


In [3]:
# pip install pycaret

In [4]:
# pip install -r requirements.txt

In [5]:
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime
from time import time
import pickle

from sklearn.metrics import roc_auc_score, accuracy_score, average_precision_score, brier_score_loss, accuracy_score, matthews_corrcoef
from sklearn.model_selection import RepeatedStratifiedKFold

# pip install pycaret
import pycaret # PyCaret
from pycaret import classification
print(f'PyCaret: version {pycaret.__version__}.')

PyCaret: version 2.3.3.


<a id='functions_classes'></a>

## Functions and classes

In [6]:
from utils import running_time, correct_col_name, train_test_split
from pre_process import pre_process

<a id='settings'></a>

## Settings

### Data management

In [7]:
# Identification of the test:
estimation_id = str(int(time()))

# Declare whether to export results:
export = True

### ML pipeline search

#### Data processing parameters

In [8]:
# Name of the response variable:
target = 'class'

# Whether data should be preprocessed or have already been treated:
preprocess = True

# Whether to apply PCA:
pca = False

# Whether to apply polynomial transformation:
polynomial_features = False

# Whether to implement features selection:
feature_selection = True

#### Search complexity parameters

In [9]:
# Time budget (minutes) for the algorithm to finally select the best models:
budget_time = 6*60

#### Estimation parameters

In [10]:
# Validation strategy:
fold_strategy, fold = ('kfold', 5)

# Performance metric of reference:
metric = 'AUC'

# Number of models to select:
n_select = 5

#### Computation parameters

In [11]:
# Number of processes to use in parallel:
n_jobs = -1

# Whether to use GPU or CPU:
use_gpu = False

<a id='imports'></a>

## Importing datasets

<a id='feats_outcomes'></a>

### Features and outcome variables

In [12]:
# Importing data:
df = pd.read_csv('../Datasets/Android_Permission.csv')

# Columns names:
df.columns = [correct_col_name(c) for c in df.columns]

# Auxiliary variables:
drop_vars = ['app', 'package', 'class']

# Removing duplicates:
df.drop_duplicates(inplace=True)

print(f'Shape of data: {df.shape}.')
df.head(3)

Shape of data: (27310, 184).


,app,package,category,description,rating,number_of_ratings,price,related_apps,dangerous_permissions_count,safe_permissions_count,access_drm_content_,access_email_provider_data,access_all_system_downloads,access_download_manager_,advanced_download_manager_functions_,audio_file_access,install_drm_content_,modify_google_service_configuration,modify_google_settings,move_application_resources,read_google_settings,send_download_notifications_,voice_search_shortcuts,access_surfaceflinger,access_checkin_properties,access_the_cache_filesystem,access_to_passwords_for_google_accounts,act_as_an_account_authenticator,bind_to_a_wallpaper,bind_to_an_input_method,change_screen_orientation,coarse,control_location_update_notifications,control_system_backup_and_restore,delete_applications,delete_other_applications_caches,delete_other_applications_data,directly_call_any_phone_numbers,directly_install_applications,disable_or_modify_status_bar,discover_known_accounts,display_unauthorized_windows,enable_or_disable_application_components,force_application_to_close,force_device_reboot,full_internet_access,interact_with_a_device_admin,manage_application_tokens,mock_location_sources_for_testing,modify_battery_statistics,modify_secure_system_settings,modify_the_google_services_map,modify_delete_usb_storage_contents_modify_delete_sd_card_contents,monitor_and_control_all_application_launching,partial_shutdown,permanently_disable_device,permission_to_install_a_location_provider,power_device_on_or_off,press_keys_and_control_buttons,prevent_app_switches,read_frame_buffer,read_instant_messages,read_phone_state_and_identity,record_what_you_type_and_actions_you_take,reset_system_to_factory_defaults,run_in_factory_test_mode,set_time,set_wallpaper_size_hints,start_im_service,update_component_usage_statistics,write_contact_data,write_instant_messages,development_tools_enable_application_debugging,development_tools_limit_number_of_running_processes,development_tools_make_all_background_applications_close,development_tools_send_linux_signals_to_applications,hardware_controls_change_your_audio_settings,hardware_controls_control_flashlight,hardware_controls_control_vibrator,hardware_controls_record_audio,hardware_controls_take_pictures_and_videos,hardware_controls_test_hardware,network_communication_broadcast_data_messages_to_applications_,network_communication_control_near_field_communication,network_communication_create_bluetooth_connections,network_communication_download_files_without_notification,network_communication_full_internet_access,network_communication_make_receive_internet_calls,network_communication_receive_data_from_internet,network_communication_view_wi_fi_state,network_communication_view_network_state,phone_calls_intercept_outgoing_calls,phone_calls_modify_phone_state,phone_calls_read_phone_state_and_identity,services_that_cost_you_money_directly_call_phone_numbers,services_that_cost_you_money_send_sms_messages,storage_modify_delete_usb_storage_contents_modify_delete_sd_card_contents,system_tools_allow_wi_fi_multicast_reception,system_tools_automatically_start_at_boot,system_tools_bluetooth_administration,system_tools_change_wi_fi_state,system_tools_change_background_data_usage_setting,system_tools_change_network_connectivity,system_tools_change_your_ui_settings,system_tools_delete_all_application_cache_data,system_tools_disable_keylock,system_tools_display_system_level_alerts,system_tools_expand_collapse_status_bar,system_tools_force_stop_other_applications,system_tools_format_external_storage,system_tools_kill_background_processes,system_tools_make_application_always_run,system_tools_measure_application_storage_space,system_tools_modify_global_animation_speed,system_tools_modify_global_system_settings,system_tools_mount_and_unmount_filesystems,system_tools_prevent_device_from_sleeping,system_tools_read_subscribed_feeds,system_tools_read_sync_settings,system_tools_read_sync_statistics,system_tools_read_write_to_resources_owned_by_diag,system_tools_reo

#### Features names

In [13]:
features_names = list(df.drop(drop_vars, axis=1).columns)

for i in range(len(features_names)):
  print(features_names[i:i+10], '\n')

['category', 'description', 'rating', 'number_of_ratings', 'price', 'related_apps', 'dangerous_permissions_count', 'safe_permissions_count', 'access_drm_content_', 'access_email_provider_data'] 

['description', 'rating', 'number_of_ratings', 'price', 'related_apps', 'dangerous_permissions_count', 'safe_permissions_count', 'access_drm_content_', 'access_email_provider_data', 'access_all_system_downloads'] 

['rating', 'number_of_ratings', 'price', 'related_apps', 'dangerous_permissions_count', 'safe_permissions_count', 'access_drm_content_', 'access_email_provider_data', 'access_all_system_downloads', 'access_download_manager_'] 

['number_of_ratings', 'price', 'related_apps', 'dangerous_permissions_count', 'safe_permissions_count', 'access_drm_content_', 'access_email_provider_data', 'access_all_system_downloads', 'access_download_manager_', 'advanced_download_manager_functions_'] 

['price', 'related_apps', 'dangerous_permissions_count', 'safe_permissions_count', 'access_drm_content_

#### Data types

In [14]:
data_types = pd.DataFrame(df.dtypes, columns=['type']).reset_index(drop=False)
data_types.columns = ['feature', 'type']

print('\033[1mDistribution of data types:\033[0m')
print(data_types.type.value_counts())

Distribution of data types:
int64      176
object       5
float64      3
Name: type, dtype: int64


#### Train-test split

In [15]:
df_train, df_test = train_test_split(df, test_ratio=0.25, shuffle=True)

#### Feature engineering

Related apps

In [16]:
# Creating the variable with the number of related apps:
df_train['related_apps'] = df_train['related_apps'].apply(lambda x: x if pd.isna(x) else x.replace('{', '').replace('}', ''))
df_train['num_related_apps'] = df_train.related_apps.apply(lambda x: x if pd.isna(x) else len(x.split(',')))
df_test['num_related_apps'] = df_test.related_apps.apply(lambda x: x if pd.isna(x) else len(x.split(',')))

# Updating the list of auxiliary variables:
drop_vars.append('related_apps')

Description

In [17]:
# Creating the variable that indicates the number of words in a description:
df_train['num_words_desc'] = df_train.description.apply(lambda x: x if pd.isna(x) else len(x.split(' ')))
df_test['num_words_desc'] = df_test.description.apply(lambda x: x if pd.isna(x) else len(x.split(' ')))

# Updating the list of auxiliary variables:
drop_vars.append('description')

Category

Even though this feature engineering is actually a transformation applied over categorical features, we first implement one-hot encoding in order to translate this categorical attribute into a numerical one, since some AutoML tools explored within this project do not allow textual inputs.

In [18]:
from transformations import applying_one_hot

In [19]:
transf_data = applying_one_hot(training_data=df_train, cat_vars=['category'], variance_param=-1, test_data=df_test)
df_train = transf_data['training_data']
df_test = transf_data['test_data']

Number of categorical features: 1
Number of overall selected dummies: 30.


In [20]:
# df_train, df_test, df_train_scaled, df_test_scaled = pre_process(training_data=df_train, test_data=df_test,
#                                                                  vars_to_drop=drop_vars,
#                                                                  log_transform=True, standardize=True)

<a id='data_description'></a>

## Data description

<a id='features_types'></a>

### Features types

In [21]:
feature_types = pd.DataFrame(df_train.drop(drop_vars, axis=1).dtypes, columns=['type']).reset_index(drop=False)
feature_types.columns = ['feature', 'type']

print('\033[1mDistribution of data types (features):\033[0m')
print(feature_types.type.value_counts())

Distribution of data types (features):
int64      175
uint8       30
float64      5
Name: type, dtype: int64


<a id='ml_pipeline'></a>

## ML pipeline

In [22]:
help(classification.setup)

Help on function setup in module pycaret.classification:

setup(data: pandas.core.frame.DataFrame, target: str, train_size: float = 0.7, test_data: Union[pandas.core.frame.DataFrame, NoneType] = None, preprocess: bool = True, imputation_type: str = 'simple', iterative_imputation_iters: int = 5, categorical_features: Union[List[str], NoneType] = None, categorical_imputation: str = 'constant', categorical_iterative_imputer: Union[str, Any] = 'lightgbm', ordinal_features: Union[Dict[str, list], NoneType] = None, high_cardinality_features: Union[List[str], NoneType] = None, high_cardinality_method: str = 'frequency', numeric_features: Union[List[str], NoneType] = None, numeric_imputation: str = 'mean', numeric_iterative_imputer: Union[str, Any] = 'lightgbm', date_features: Union[List[str], NoneType] = None, ignore_features: Union[List[str], NoneType] = None, normalize: bool = False, normalize_method: str = 'zscore', transformation: bool = False, transformation_method: str = 'yeo-johnson', 

In [23]:
help(classification.compare_models)

Help on function compare_models in module pycaret.classification:

compare_models(include: Union[List[Union[str, Any]], NoneType] = None, exclude: Union[List[str], NoneType] = None, fold: Union[int, Any, NoneType] = None, round: int = 4, cross_validation: bool = True, sort: str = 'Accuracy', n_select: int = 1, budget_time: Union[float, NoneType] = None, turbo: bool = True, errors: str = 'ignore', fit_kwargs: Union[dict, NoneType] = None, groups: Union[str, Any, NoneType] = None, verbose: bool = True) -> Union[Any, List[Any]]
    This function trains and evaluates performance of all estimators available in the 
    model library using cross validation. The output of this function is a score grid 
    with average cross validated scores. Metrics evaluated during CV can be accessed 
    using the ``get_metrics`` function. Custom metrics can be added or removed using 
    ``add_metric`` and ``remove_metric`` function.
    
    Example
    -------
    >>> from pycaret.datasets import get_da

<a id='ml_pipeline_search'></a>

### ML pipeline search

#### Preparing the data

In [24]:
# Describing and pre-processing the data:
setup = classification.setup(data=df_train.drop([v for v in drop_vars if v!='class'], axis=1),
                             test_data=df_test.drop([v for v in drop_vars if v!='class'], axis=1),

                             # Data processing parameters:
                             preprocess=preprocess, pca=pca, polynomial_features=polynomial_features, feature_selection=feature_selection,
                             target=target, session_id=int(estimation_id),

                             # Estimation parameters:
                             fold_strategy=fold_strategy, fold=fold,

                             # Computation parameters:
                             n_jobs=n_jobs, use_gpu=use_gpu
                             )

,Description,Value
0,session_id,1631225326
1,Target,class
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(20483, 211)"
5,Missing Values,True
6,Numeric Features,35
7,Categorical Features,175
8,Ordinal Features,False
9,High Cardinality Features,False


#### Setting and running the search

In [25]:
start_time = datetime.now()

# Searching across a collection of different ML algorithms:
best_models = classification.compare_models(
    # Search complexity parameters:
    budget_time=budget_time,

    # Estimation parameters:
    sort=metric, n_select=n_select
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.8128,0.9013,0.8291,0.8829,0.8551,0.5915,0.5939,0.594
xgboost,Extreme Gradient Boosting,0.8082,0.8939,0.8348,0.8720,0.8529,0.5776,0.5788,15.958
gbc,Gradient Boosting Classifier,0.8075,0.8908,0.8378,0.8686,0.8529,0.5746,0.5755,5.362
ada,Ada Boost Classifier,0.8066,0.8879,0.8520,0.8569,0.8544,0.5662,0.5664,1.682
rf,Random Forest Classifier,0.7857,0.8560,0.8419,0.8374,0.8396,0.5167,0.5169,3.744
lda,Linear Discriminant Analysis,0.7416,0.8159,0.8633,0.7747,0.8165,0.3839,0.3906,0.892
et,Extra Trees Classifier,0.7627,0.7694,0.8281,0.8182,0.8230,0.4633,0.4636,5.332
nb,Naive Bayes,0.5975,0.7218,0.4639,0.8787,0.6009,0.2663,0.3269,0.120
knn,K Neighbors Classifier,0.6804,0.7151,0.7835,0.7486,0.7656,0.2642,0.2650,1.636
dt,Decision Tree Classifier,0.7391,0.7066,0.8058,0.8033,0.8045,0.4124,0.4125,0.382


In [26]:
# Training the best model:
model = classification.stack_models(estimator_list=best_models, optimize=metric) # For training a model stacking.
# model = classification.create_model(estimator='gbc') # If only one model is expected to be trained.

# Total elapsed time:
end_time = datetime.now()
search_time = running_time(start_time=start_time, end_time=end_time)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7242,0.7982,0.8573,0.7607,0.8061,0.3344,0.3416
1,0.7857,0.8580,0.8477,0.8361,0.8419,0.5096,0.5097
2,0.7876,0.8637,0.8491,0.8344,0.8417,0.5193,0.5195
3,0.8093,0.8972,0.8499,0.8560,0.8529,0.5819,0.5819
4,0.8044,0.8889,0.8287,0.8747,0.8511,0.5669,0.5687
Mean,0.7823,0.8612,0.8465,0.8324,0.8387,0.5024,0.5043
SD,0.0305,0.0348,0.0095,0.0387,0.0170,0.0884,0.0859


------------------------------------
Running time: 16.0 minutes.
Start time: 2021-09-09, 22:10:56
End time: 2021-09-09, 22:26:56
------------------------------------


In [27]:
# Fine tuning hyperparameters:
model = classification.tune_model(estimator=model, optimize=metric)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7747,0.8486,0.8416,0.8250,0.8332,0.4862,0.4865
1,0.7857,0.8580,0.8480,0.8359,0.8419,0.5094,0.5095
2,0.8196,0.9065,0.8576,0.8694,0.8634,0.5979,0.5980
3,0.7793,0.8588,0.8540,0.8155,0.8343,0.5044,0.5056
4,0.8044,0.8889,0.8287,0.8747,0.8511,0.5669,0.5687
Mean,0.7928,0.8722,0.8460,0.8441,0.8448,0.5330,0.5337
SD,0.0168,0.0219,0.0102,0.0238,0.0113,0.0423,0.0424


In [28]:
# Training the final model (including the hold-out data):
final_model = classification.finalize_model(estimator=model)

### Assessing the outcomes

#### ML pipeline

Constructed pipelines

After running the search with *compare_models* function, the complete list of tested models is presented together with statistics of reference.

Final pipeline

In [29]:
# Saving and presenting the selected ML pipeline:
classification.save_model(final_model, f'../Datasets/Outcomes/pycaret/ML_pipeline_{estimation_id}')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='class',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strate...
                                                                         warm_start=False))],
                                     final_estimator=LogisticRegression(C=1.078,
                                                                        class_weight={},
          

#### Model evaluation

In [30]:
# Visualizing the model evaluation on test data:
classification.plot_model(estimator=model, plot='auc', save=True)

'AUC.png'

In [35]:
# # Predictions for hold-out data:
classification.predict_model(estimator=model)

# Performance metrics of the best model:
test_roc_auc = 0.8593
test_avg_prec = 0.8409
test_acc = 0.7901
test_mcc = 0.5223

print(f'Test ROC-AUC: {test_roc_auc:.4f}.')
print(f'Test average-precision score: {test_avg_prec:.4f}.')
print(f'Test accuracy: {test_acc:.4f}.')
print(f'Test MCC: {test_mcc:.4f}.')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Stacking Classifier,0.7901,0.8593,0.8477,0.8409,0.8443,0.5222,0.5223


Test ROC-AUC: 0.8593.
Test average-precision score: 0.8409.
Test accuracy: 0.7901.
Test MCC: 0.5223.


#### Model assessment

In [32]:
model_assess = {
    "estimation_id": estimation_id,
    "autoML": "pycaret",
    "parameters": {
      "pre_proc": {
        "target": target, "preprocess": preprocess, "pca": pca, "polynomial_features": polynomial_features, "feature_selection": feature_selection
      },
      "search_complexity": {
        "budget_time": budget_time
      },
      "estimation": {
        "fold_strategy": fold_strategy, "fold": fold, "metric": metric, "n_select": n_select
      },
      "computation": {
        "n_jobs": n_jobs, "use_gpu": use_gpu
      }
    },
    "running_time": search_time,
    "performance_metrics": {
        "test_roc_auc": test_roc_auc, "test_avg_prec": test_avg_prec, "test_acc": test_acc, "test_mcc": test_mcc
    }
}

### Exporting the outcomes

In [33]:
if export:
  # pickle.dump(model, open(f'../Datasets/Outcomes/pycaret/model_{estimation_id}.pickle', 'wb'))

  # Model assessment:
  with open(f'../Datasets/Outcomes/pycaret/model_assess_{estimation_id}.json', 'w') as json_file:
    json.dump(model_assess, json_file, indent=2)